# Neuronal Networks

## Set up Tensorflow

In [1]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.18.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Build a machine learning model

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


`Sequential` is useful for staking layers where each layer has one input `tensor` and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.09420404, -0.43470845, -0.24208869,  0.3612834 ,  0.0647642 ,
        -0.3003919 , -0.18934788,  0.24547103,  0.3009249 ,  0.06177798]],
      dtype=float32)

The `tf.nn.softmax` function converts these logist to probabilities for each class:

In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.0900578 , 0.06406815, 0.07767761, 0.14201592, 0.10557483,
        0.07327825, 0.08188435, 0.12648539, 0.13369761, 0.10526003]],
      dtype=float32)

Note: it is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [10]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.6134915)

Before you start training, configure and compile the model using Keras `Model.compile`. Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [11]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8605 - loss: 0.4783
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9555 - loss: 0.1490
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9659 - loss: 0.1090
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9746 - loss: 0.0826
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9783 - loss: 0.0705


In [13]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9765 - loss: 0.0765


[0.07646095752716064, 0.9764999747276306]

In [14]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [15]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.3802417e-07, 6.8122707e-10, 9.6974800e-05, 6.2830797e-05,
        1.6841946e-10, 3.8955662e-08, 5.8014840e-13, 9.9983108e-01,
        1.0635636e-06, 7.7520190e-06],
       [1.2040912e-08, 9.3704475e-05, 9.9990201e-01, 2.0435935e-07,
        9.9230291e-15, 2.3701881e-08, 3.9022311e-06, 1.7820202e-13,
        8.2547857e-08, 7.7031881e-16],
       [5.6427592e-07, 9.9825865e-01, 4.0181010e-04, 8.2560946e-06,
        1.0201243e-04, 4.6863415e-06, 5.9381786e-05, 1.0178669e-03,
        1.4496608e-04, 1.8461936e-06],
       [9.9973577e-01, 1.1896657e-08, 2.6754378e-05, 3.7973015e-07,
        4.6059949e-06, 5.1384623e-06, 1.4534128e-04, 7.8666460e-05,
        3.1915416e-08, 3.3512079e-06],
       [3.4748977e-05, 5.1199333e-08, 4.5421307e-06, 3.8600157e-07,
        9.6850991e-01, 3.9412447e-07, 1.6762371e-05, 6.6160232e-05,
        2.7612043e-06, 3.1364325e-02]], dtype=float32)>

In [ ]:
# from https://www.tensorflow.org/tutorials/quickstart/beginner